# Vertical FL Example

In [ ]:
!pip install git+https://github.com/securefederatedai/openfl.git
!pip install -r ../workflow_interface_requirements.txt

In [ ]:
from openfl.experimental.interface import FLSpec, Aggregator, Collaborator
from openfl.experimental.runtime import LocalRuntime
from openfl.experimental.placement import aggregator, collaborator

In [ ]:
class VerticalFlow(FLSpec):

    @aggregator
    def start(self):
        self.collaborators = self.runtime.collaborators
        self.round = 0
        self.next_collaborator = ['Portland']
        self.next(self.custom_task_portland, foreach='next_collaborator')

    @collaborator
    def custom_task_portland(self):
        print(f'Collaborator {self.input}: performing custom task')
        self.result = 0
        self.next(self.gather_portland_results)

    @aggregator
    def gather_portland_results(self,inputs):
        self.results = []
        self.results.append(inputs[0].result)
        self.next_collaborator = ['Seattle']
        self.next(self.custom_task_seattle, foreach='next_collaborator', exclude=['results'])

    @collaborator
    def custom_task_seattle(self):
        print(f'Collaborator {self.input}: performing custom task')
        self.result = 1
        self.next(self.gather_seattle_results)

    @aggregator
    def gather_seattle_results(self,inputs):
        self.results.append(inputs[0].result)
        self.next_collaborator = ['Chandler']
        self.next(self.custom_task_chandler, foreach='next_collaborator', exclude=['results'])

    @collaborator
    def custom_task_chandler(self):
        print(f'Collaborator {self.input}: performing custom task')
        self.result = 2
        self.next(self.gather_chandler_results)

    @aggregator
    def gather_chandler_results(self,inputs):
        self.results.append(inputs[0].result)
        self.next_collaborator = ['Bangalore']
        self.next(self.custom_task_bangalore, foreach='next_collaborator', exclude=['results'])

    @collaborator
    def custom_task_bangalore(self):
        print(f'Collaborator {self.input}: performing custom task')
        self.result = 3
        self.next(self.gather_bangalore_results)

    @aggregator
    def gather_bangalore_results(self,inputs):
        self.results.append(inputs[0].result)
        self.next(self.combine)

    @aggregator
    def combine(self):
        print(f'The results from each of the collaborators are: {self.results}')
        print(f'Their average = {sum(self.results) / len(self.results)}')
        self.round += 1
        if self.round < 10:
            print()
            print(f'Starting round {self.round}...')
            self.next_collaborator = ['Portland']
            self.next(self.custom_task_portland,foreach='next_collaborator')
        else:
            self.next(self.end)

    @aggregator
    def end(self):
        print(f'This is the end of the flow')

In [ ]:
# Setup participants
aggregator = Aggregator()

# Setup collaborators private attributes via callable function
collaborator_names = ['Portland', 'Seattle', 'Chandler', 'Bangalore']
collaborators = []
for idx, collaborator_name in enumerate(collaborator_names):
    collaborators.append(Collaborator(name=collaborator_name))

local_runtime = LocalRuntime(
    aggregator=aggregator, collaborators=collaborators,backend='ray')
print(f'Local runtime collaborators = {local_runtime.collaborators}')

vflow = VerticalFlow(checkpoint=True)
vflow.runtime = local_runtime
vflow.run()
print(f'Reached end of the flow with collaborator results = {vflow.results}')

Now that the flow has completed, you can use metaflow tooling to look at the data associated with the experiment

In [ ]:
run_id = vflow._run_id

In [ ]:
from metaflow import Metaflow, Flow, Step, Task

In [ ]:
m = Metaflow()
list(m)

In [ ]:
f = Flow('VerticalFlow').latest_run

In [ ]:
f

In [ ]:
list(f)

In [ ]:
s = Step(f'VerticalFlow/{run_id}/combine')

In [ ]:
s

In [ ]:
list(s)

In [ ]:
t = Task(f'VerticalFlow/{run_id}/combine/91')

In [ ]:
t

In [ ]:
t.data

In [ ]:
t.data.round